# Get all features together with the evaluation times for all queries (DuckDB)

Features based on the query:   
*  number of relations
*  number of conditions
*  number of filters
*  number of joins

Features based on the join tree:
*  depth
*  container count (min, max, mean, median, q1, q3)
*  branching factors (min, max, mean, median, q1, q3)
*  balancedness factor 

Features based on the data in the database (EXPLAIN):
*  estimated cardinalities (of tables, filters, joins) (min, max, mean, median, q1, q3)

In [1]:
%%bash
pip install pandas
pip install duckdb
pip install psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 97.5 MB/s eta 0:00:00


In [2]:
import csv
import re
import pandas as pd
import duckdb
import psycopg2
import numpy as np
import os
import json

### Get the features based on the structure of the query and database information

In [3]:
def iterate_through_plan(plan, table_rows, join_rows):
    if plan["Node Type"] == "Seq Scan":
        table_rows.append(plan["Plan Rows"])
    elif plan["Node Type"] == "Index Only Scan":
        table_rows.append(plan["Plan Rows"])
    elif plan["Node Type"] == "Index Scan":
        table_rows.append(plan["Plan Rows"])
    elif plan["Node Type"] == "Bitmap Index Scan":
        table_rows.append(plan["Plan Rows"])
    
    if plan["Node Type"] == "Hash Join":
        join_rows.append(plan["Plan Rows"])
    elif plan["Node Type"] == "Merge Join":
        join_rows.append(plan["Plan Rows"])
    elif plan["Node Type"] == "Nested Loop":
        join_rows.append(plan["Plan Rows"])

    if "Plans" in plan.keys():
        for i in range(len(plan["Plans"])):
            iterate_through_plan(plan["Plans"][i], table_rows, join_rows)

    return table_rows, join_rows

In [4]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

1. features only of the query structure:

In [5]:
# Define input and output file paths
input_file = 'scala_commands_augment_filter_agg.txt'
output_file = 'results/featuresDatabase_DDB.csv'

# Open input and output files
with open(input_file, 'r') as f_input, open(output_file, 'w', newline='') as f_output:
    csv_writer = csv.writer(f_output)
    
    # Write header to CSV file
    csv_writer.writerow(['bench', 'query', '#relations', '#conditions', '#filters', '#joins', 'text'])

    database_old = " "
    # Read input file line by line
    for line in f_input:
        # Split each line into components
        pattern = r'(?<!\\)\"|\"(?<!\\)(?=\s+\"|$)'
        components = re.split(pattern, line)
        
        # Extract relevant information
        benchmark = components[3]
        number = components[5]
        query = components[1].strip()

        # FEATURES BASED ON QUERY STRUCTURE
        # get the number of relations
        query_upper = query.upper()
        from_index = query_upper.find("FROM")
        where_index = query_upper.find("WHERE")
        number_of_relations = query[from_index:where_index].count(",") + 1

        # get the number of conditions
        number_of_conditions = query.count("AND") + 1

        # get how many filter and join conditions
        parts = query_upper.split("WHERE")[1].split("AND")
        filter = 0
        join = 0
        joins = []
        for p in parts:
            partners = []
            p_split = p.split("=")
            if len(p_split) == 2 and p_split[1].count("'") == 0 and p_split[1].count('"') == 0 and is_number(p_split[1].strip()) == False:
                partners = [p2.strip().split(".")[0] for p2 in p.split("=")]
                if partners not in joins and list(reversed(partners)) not in joins:
                    joins.append(partners)
                    join += 1
            else:
                filter += 1

        # Write data to CSV file
        csv_writer.writerow([benchmark, number, number_of_relations, number_of_conditions, filter, join, query])

2. features of the query structure and of DuckDB EXPLAIN:

In [6]:
# Define input and output file paths
input_file = 'scala_commands_augment_filter_agg.txt'
output_file = 'results/featuresDatabase_DDB_extra.csv'

# Open input and output files
with open(input_file, 'r') as f_input, open(output_file, 'w', newline='') as f_output:
    csv_writer = csv.writer(f_output)
    
    # Write header to CSV file
    csv_writer.writerow(['bench', 'query', '#relations', '#conditions', '#filters', '#joins', 'min(est. cardinality)', 'max(est. cardinality)', 
                         'mean(est. cardinality)', 'q25(est. cardinality)', 'median(est. cardinality)', 'q75(est. cardinality)',
                         'list est. cardinality', 'text'])

    database_old = " "
    # Read input file line by line
    for line in f_input:
        # Split each line into components
        pattern = r'(?<!\\)\"|\"(?<!\\)(?=\s+\"|$)'
        components = re.split(pattern, line)
        
        # Extract relevant information
        benchmark = components[3]
        number = components[5]
        query = components[1].strip()

        # FEATURES BASED ON QUERY STRUCTURE
        # get the number of relations
        query_upper = query.upper()
        from_index = query_upper.find("FROM")
        where_index = query_upper.find("WHERE")
        number_of_relations = query[from_index:where_index].count(",") + 1

        # get the number of conditions
        number_of_conditions = query.count("AND") + 1

        # get how many filter and join conditions
        parts = query_upper.split("WHERE")[1].split("AND")
        filter = 0
        join = 0
        joins = []
        for p in parts:
            partners = []
            p_split = p.split("=")
            if len(p_split) == 2 and p_split[1].count("'") == 0 and p_split[1].count('"') == 0 and is_number(p_split[1].strip()) == False:
                partners = [p2.strip().split(".")[0] for p2 in p.split("=")]
                if partners not in joins and list(reversed(partners)) not in joins:
                    joins.append(partners)
                    join += 1
            else:
                filter += 1

        # FEATURES BASED ON DATABASE
        # get features based on the DuckDB Plan by EXPLAIN
        database = benchmark.lower() + "/" + benchmark.lower() + ".duckdb"
        if database != database_old:
            if benchmark == "JOB":
                database_pos = "imdb"
            else:
                database_pos = benchmark.lower()
            conn = psycopg2.connect(
                    host="postgres",
                    database=database_pos,
                    user=database_pos,
                    password=database_pos)
            cur = conn.cursor()
            cur.execute("SELECT inet_server_addr(), inet_server_port()")
            host, port = cur.fetchone()
            cur.close()
            conn.close()
            con = duckdb.connect(database=database)
            con.execute("INSTALL postgres")
            con.execute("LOAD postgres")
            con.execute("ATTACH 'host=" + host + " port=5432 user=postgres password=postgres dbname=" + database_pos + "' AS " + 
                        benchmark.lower() + "_DDB (TYPE postgres)")
            con.execute("USE " + benchmark.lower() + "_DDB")
            database_old = database

        conn = duckdb.connect(database=database)
        cur = conn.cursor()
        cur.execute("USE " + benchmark.lower() + "_DDB")
        query2 = query.replace('\\\\\\"', '')
        cur.execute("EXPLAIN " + query2)
        result = cur.fetchall()

        pattern = r'EC:\s*(\w+)'
        est_card = re.findall(pattern, result[0][1])

        est_card = list(map(int, est_card))
        # calculate min, max, mean, median, 0.25-quantile and 0.75-quantile
        est_card_min = np.min(est_card)
        est_card_max = np.max(est_card)
        est_card_median = np.median(est_card)
        est_card_mean = np.mean(est_card)
        est_card_q25 = np.quantile(est_card, 0.25)
        est_card_q75 = np.quantile(est_card, 0.75)
                
        # Write data to CSV file
        csv_writer.writerow([benchmark, number, number_of_relations, number_of_conditions, filter, join, est_card_min, est_card_max, 
                             est_card_mean, est_card_q25, est_card_median, est_card_q75, est_card, query])

### Get the features based on the join tree structure (calculated in Scala, imported and formated here)

In [7]:
output_file = 'results/featuresScala_DDB.csv'
csv_header = ["bench", "query", "depth", "min(container counts)", "max(container counts)", "mean(container counts)", "q25(container counts)",
              "median(container counts)", "q75(container counts)", "min(branching factors)", "max(branching factors)", "mean(branching factors)", 
              "median(branching factors)", "q25(branching factors)", "q75(branching factors)", "balancedness factor", "container counts list", 
              "branching factors list"]
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(csv_header)

    directory = 'rewritten/'
    for filename in os.listdir(directory):
        if filename.endswith("output.json"):
            filepath = os.path.join(directory, filename)
            benchmark = filename.split("_")[0]
            number = filename.split("_")[1]
            with open(filepath, 'r') as file:
                data = json.load(file)
                # FEATURES BASED ON THE JOIN TREE STRUCTURE
                feature = data.get("features", [])
                features = feature.split("List(")
                depth = int(features[1][:-2])
                container_counts = [int(x) for x in features[2][:-3].split(", ")]
                container_counts_min = np.min(container_counts)
                container_counts_max = np.max(container_counts)
                container_counts_mean = np.mean(container_counts)
                container_counts_median = np.median(container_counts)
                container_counts_q25 = np.quantile(container_counts, 0.25)
                container_counts_q75 = np.quantile(container_counts, 0.25)
                branching_factors = [int(x) for x in features[3].split("), ")[0].split(", ")]
                branching_factors_min = np.min(branching_factors)
                branching_factors_max = np.max(branching_factors)
                branching_factors_mean = np.mean(branching_factors)
                branching_factors_median = np.median(branching_factors)
                branching_factors_q25 = np.quantile(branching_factors, 0.25)
                branching_factors_q75 = np.quantile(branching_factors, 0.75)
                balancedness_factor = float(features[3].split("), ")[1][:-1])

                # Write data to CSV file
                csv_row = [benchmark, number, depth, container_counts_min, container_counts_max, container_counts_mean, container_counts_q25,
                           container_counts_median, container_counts_q75, branching_factors_min, branching_factors_max, branching_factors_mean,
                           branching_factors_median, branching_factors_q25, branching_factors_q75, balancedness_factor, container_counts, 
                           branching_factors]
                writer.writerow(csv_row)

### Merge all features and evaluation times for each query

1. Merge features of the query structure and the join tree with the evaluation time:

In [8]:
df1 = pd.read_csv('results/featuresDatabase_DDB.csv')
df2 = pd.read_csv('results/featuresScala_DDB.csv')
df3 = pd.read_csv('results/DDB_Scala_comparison_TO_augment_server.csv')
df3['orig mean'] = df3['orig mean'].replace('TO', 100).astype("float64")
df3['rewr mean+rewr'] = df3['rewr mean+rewr'].replace('TO', 100).astype("float64")
df3['rewr mean'] = df3['rewr mean'].replace('TO', 100).astype("float64")
df3['diff rewr+rewr-orig'] = df3['rewr mean+rewr'] - df3['orig mean']
df3['diff rewr-orig'] = df3['rewr mean'] - df3['orig mean']

merged_df = pd.merge(df1, df2, on=["bench", "query"], how='inner').merge(df3, on=["bench", "query"], how='inner')

2. Merge features of the query structure, the join tree and the DuckDB EXPLAIN with the evaluation time:

In [9]:
df1_extra = pd.read_csv('results/featuresDatabase_DDB_extra.csv')
df2_extra = pd.read_csv('results/featuresScala_DDB.csv')
df3_extra = pd.read_csv('results/DDB_Scala_comparison_TO_augment_server.csv')
df3_extra['orig mean'] = df3_extra['orig mean'].replace('TO', 100).astype("float64")
df3_extra['rewr mean+rewr'] = df3_extra['rewr mean+rewr'].replace('TO', 100).astype("float64")
df3_extra['rewr mean'] = df3_extra['rewr mean'].replace('TO', 100).astype("float64")
df3_extra['diff rewr+rewr-orig'] = df3_extra['rewr mean+rewr'] - df3_extra['orig mean']
df3_extra['diff rewr-orig'] = df3_extra['rewr mean'] - df3_extra['orig mean']

merged_df_extra = pd.merge(df1_extra, df2_extra, on=["bench", "query"], how='inner').merge(df3_extra, on=["bench", "query"], how='inner')

### Save the resulting dataframes as csv

In [10]:
merged_df[["bench", "query", "orig/rewr(mean)", "orig/rewr+rewr(mean)", "orig mean", "rewr mean", "rewr mean+rewr", "diff rewr-orig", 
           "diff rewr+rewr-orig", "#relations", "#conditions", "#filters", "#joins", "depth", "min(container counts)", "max(container counts)", 
           "mean(container counts)", "q25(container counts)", "median(container counts)", "q75(container counts)", "min(branching factors)", 
           "max(branching factors)", "mean(branching factors)",  "median(branching factors)", "q25(branching factors)", "q75(branching factors)", 
           "balancedness factor", "container counts list", "branching factors list", "text"]].to_csv('results/features_times_DDB.csv', index=False)

In [11]:
merged_df_extra[["bench", "query", "orig/rewr(mean)", "orig/rewr+rewr(mean)", "orig mean", "rewr mean", "rewr mean+rewr", "diff rewr-orig", 
           "diff rewr+rewr-orig", "#relations", "#conditions", "#filters", "#joins", 'min(est. cardinality)', 'max(est. cardinality)', 
           'mean(est. cardinality)', 'q25(est. cardinality)', 'median(est. cardinality)', 'q75(est. cardinality)', "depth", "min(container counts)", 
           "max(container counts)", "mean(container counts)", "q25(container counts)", "median(container counts)", "q75(container counts)", 
           "min(branching factors)", "max(branching factors)", "mean(branching factors)", "median(branching factors)", "q25(branching factors)", 
           "q75(branching factors)", "balancedness factor", 'list est. cardinality', "container counts list", "branching factors list",
           "text"]].to_csv('results/features_times_DDB_extra.csv', index=False)